<a href="https://colab.research.google.com/github/ucbnlp24/hws4nlp24/blob/main/HW1/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [162]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Homework 1: Featurized Models for Sentiment Analysis

## Introduction

In this homework, you will write code that generates features for the positive/negative sentiment classification of movie
reviews, and train a model on those features to predict the sentiment of new data. Example of selected movie reviews
with labels:

| Label | Review
| :--- | :------
| **pos** | What can I say, it’s a damn good movie. See it if you still haven’t. Great camera works and lighting <br> techniques. Awesome, just awesome.  Orson Welles is incredible ‘The Lady From Shanghai’ can certainly <br> take the place of ‘Citizen Kane’.
| **neg** | Can I give this a minus rating? No? Well, let me say that this is the most atrocious film I have ever tried to <br> watch. It was Painful. Boringus Maximus. The plot(?) is well hidden in several sub-levels of <br> nebulosity. I rented this film with a friend and, after about thirty minutes of hoping it would get better, we <br> decided to “fast forward” a little to see if things would get any better. It never gets better. This film about <br> some dude getting kidnapped by these two girls, sounds interesting, but, in reality, it is just a bore. <br> Nothing even remotely interesting ever happens. If you ever get the chance to watch this, do <br> yourself a favor, try “PLAN NINE FROM OUTER SPACE” instead ”  

This notebook contains code which performs all the data parsing, model training, and evaluation for you. Once you are in this notebook, **save a copy** in your Google Drive or Github before working on the file.

During training, a basic logistic regression model (from the scikit-learn package) is trained with your created features
to predict each review’s sentiment. Your ultimate goal for this assignment is to featurize the text creatively and optimize
the accuracy of the model on the test data.

Your main tasks regarding *classification* are to **implement four features**:
1. A bag of words feature. Fill out the **bag_of_words** function in the notebook to represent a document through its bag of words. Represent each **lowercase** word tokenized with the NLTK **word_tokenize** function through its binary value. Remember that the feature value for a word that shows up in a review should be 1, no matter how many times it is mentioned. Please keep your code between the **#BEGIN/#END SOLUTION** flags.
2. Create **three** different classes of features. Implement them in the **feature1** **feature2** and **feature3** functions. Create features that you think would perform better than bag of words, and assess their independent performance on the development data. Describe your reasoning in your Colab notebook and include accuracy scores on the dev.txt data (printed in the Colab output). **You must report your reasoning and development scores in the table provided.** It is not required that your features actually perform well, but your justification and reasoning should be defensible. We are looking for thoughtful insights as to why your original features *should* outperform the simple bag of words implementation.

For more feature ideas consult [SLP Ch. 5](https://web.stanford.edu/~jurafsky/slp3/5.pdf) and lecture slides, and please do keep the following constraints in mind:

* Do not use pre-trained word vectors (including static embeddings like word2vec or contextual embeddings like BERT); we will cover these later.
* Do not use the predictions of any other supervised model on this data as a feature for logistic regression (e.g., do not train a separate CNN on this data, make predictions on the dev data using the CNN, and then treat your CNN prediction as a feature).
* Do not alter {train,dev,test}.txt or find additional sources of training data. This homework is focusing on how you *represent* your data, not other aspects of the training regime.
* Do not import any other additional libraries.

After completing classificaion there is a **task** for *evaluation*:
1. Fill in the dictionary `param_grid`, to find which L2 regularization strength improves your model the most.

The assignment ends with a **reflection**:
1. *Short answer* regarding applying assignment concepts to a real world classification evaluation situation.

## How to Submit

There are two deliverables to be submitted on Gradescope. Submit to GradeScope [HW 1](https://www.gradescope.com/courses/701350/assignments/3984973):
  - Submit `HW1.ipynb`. The name of this file should stay unchanged for the Gradescope autograder.
  - Submit `combiner_function_predictions.csv`. The file must be named in this way for the Gradescope autograder.

In [1]:
##################################
## Please Do not edit the imports
##################################

from scipy import sparse
from sklearn import linear_model
from collections import Counter
import operator
import nltk
import csv
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd
from pandas import option_context

In [2]:
!python -m nltk.downloader punkt

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Intro: Gather Data + Create Classifier

### Gather Data

Let's download the data we'll use for training and development, and also the data we'll make predictions with.

In [3]:
# Get data
!wget https://raw.githubusercontent.com/ucbnlp24/hws4nlp24/main/HW1/train.txt
!wget https://raw.githubusercontent.com/ucbnlp24/hws4nlp24/main/HW1/dev.txt
!wget https://raw.githubusercontent.com/ucbnlp24/hws4nlp24/main/HW1/test.txt

--2024-02-03 20:14:04--  https://raw.githubusercontent.com/ucbnlp24/hws4nlp24/main/HW1/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1322055 (1.3M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]   1.26M  --.-KB/s    in 0.01s   

2024-02-03 20:14:05 (123 MB/s) - ‘train.txt’ saved [1322055/1322055]

--2024-02-03 20:14:05--  https://raw.githubusercontent.com/ucbnlp24/hws4nlp24/main/HW1/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1309909 (1.2M) [text/plain]
Saving to: ‘dev.t

In [4]:
trainingFile = "train.txt"
evaluationFile = "dev.txt"
testFile = "test.txt"

### Define Classifier class

Next, we've created a Binary Classifier. This class will let us learn the traits associated with positive and negatively classed movie reviews in order to make predictions on our test set.

In [90]:
######################################################################
## Do not edit this block of code.
## This defines the classification class which
## loads the data and sets up the model.
######################################################################

class Classifier:

    def __init__(self, feature_method, L2_regularization_strength=1.0, min_feature_count=1):
        # Initialize variables and set up logistic regression model
        self.feature_vocab = {} # Dictionary to store feature vocabulary
        self.feature_method = feature_method # Method to extract features from text
        self.log_reg = linear_model.LogisticRegression() # Logistic Regression model
        self.L2_regularization_strength=L2_regularization_strength # L2 regularization strength
        self.min_feature_count=min_feature_count # Minimum count for a feature to be considered

        # Process training, development, and test datasets
        self.trainX, self.trainY, self.trainOrig = self.process(trainingFile, training=True)
        self.devX, self.devY, self.devOrig = self.process(evaluationFile, training=False)
        self.testX, _, self.testOrig = self.process(testFile, training=False)

    # Read data from file and return as a list of tuples
    def load_data(self, filename):
        data = []
        with open(filename, encoding="utf8") as file:
            for line in file:
                cols = line.split("\t")
                idd = cols[0]
                label = cols[1]
                text = cols[2]

                data.append((idd, label, text))

        return data

    # Featurize entire dataset using the specified feature extraction method
    def featurize(self, data):
        featurized_data = []
        for idd, label, text in data:
            feats = self.feature_method(text)
            featurized_data.append((label, feats))
        return featurized_data

    # Read dataset and returned featurized representation as sparse matrix + label array
    def process(self, dataFile, training = False):
        original_data = self.load_data(dataFile)
        # print(f"load_data function result: {len(original_data)}")
        # print(f"first sample point outcome from load_data: {original_data[0]}\n")

        data = self.featurize(original_data)

        # for feature in data:
        #   print(len(feature[1]))
        # print(f"post featurize outcome length: {len(data)}")
        # print(f"first sample point outcome from featurize: {data[0]}\n")

        # If training, build feature vocabulary and filter features based on minimum count
        if training:
            fid = 0
            feature_doc_count = Counter()
            for label, feats in data:
                for feat in feats:
                    feature_doc_count[feat]+= 1

            for feat in feature_doc_count:
                if feature_doc_count[feat] >= self.min_feature_count:
                    self.feature_vocab[feat] = fid
                    fid += 1

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        #print(f"Sparse Matrix dimensions: {X.shape}")
        Y = [None]*D
        for idx, (label, feats) in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]
            Y[idx] = label

        return X, Y, original_data

    def load_test(self, dataFile):
        data = self.load_data(dataFile)
        data = self.featurize(data)

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        Y = [None]*D
        for idx, (data_id, feats) in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]
            Y[idx] = data_id

        return X, Y

    # Train model and evaluate on held-out data
    def evaluate(self):
        (D,F) = self.trainX.shape
        self.log_reg = linear_model.LogisticRegression(C = self.L2_regularization_strength, max_iter=1000)
        self.log_reg.fit(self.trainX, self.trainY)
        training_accuracy = self.log_reg.score(self.trainX, self.trainY)
        development_accuracy = self.log_reg.score(self.devX, self.devY)
        print("Method: %s, Features: %s, Train accuracy: %.3f, Dev accuracy: %.3f" % (self.feature_method.__name__, F, training_accuracy, development_accuracy))


    # Predict labels for new data and write the predictions to a CSV file
    def predict(self):
        predX = self.log_reg.predict(self.testX)

        with open("%s_%s" % (self.feature_method.__name__, "predictions.csv"), "w", encoding="utf8") as out:
            writer=csv.writer(out)
            writer.writerow(["Id", "Expected"])
            for idx, data_id in enumerate(self.testX):
                writer.writerow([self.testOrig[idx][0], predX[idx]])
        out.close()

    # Method to print top weights for each feature
    def printWeights(self, n=10):

        reverse_vocab=[None]*len(self.log_reg.coef_[0])
        for k in self.feature_vocab:
            reverse_vocab[self.feature_vocab[k]]=k

        # binary
        if len(self.log_reg.classes_) == 2:
              weights=self.log_reg.coef_[0]

              cat=self.log_reg.classes_[1]
              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

              cat=self.log_reg.classes_[0]
              for feature, weight in list(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1)))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

        # multiclass
        else:
          for i, cat in enumerate(self.log_reg.classes_):

              weights=self.log_reg.coef_[i]

              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

## Examples: A Simple Classifier

Let's create an initial classifier based on a really simple feature using a dictionary:

* if the abstract contains the words "love" or "like", the `contains_positive_word` feature will fire, and
* if it contains either "hate" or "dislike", the `contains_negative_word` will fire.  

Note how we use `nltk.word_tokenize` to tokenize the text into its discrete words (the documentation for which can be found [here](https://www.nltk.org/api/nltk.tokenize.html)).

In [87]:
def simple_featurize(text):
    feats = {}
    words = nltk.word_tokenize(text)

    for word in words:
        word=word.lower()
        if word == "love" or word == "like":
            feats["contains_positive_word"] = 1
        if word == "hate" or word == "dislike":
            feats["contains_negative_word"] = 1

    return feats


Now let's see how that feature performs on the development data.

Note the `L2_regularization_strength` parameter specifies the strength of the L2 regularizer **(values closer to 0 = stronger regularization)**, and `min_feature_count` specifies how many data points need to contain a feature for it to be passed into the model as a feature. Both are ways to prevent the model from overfitting and achieve higher performance.

In [88]:
simple_classifier = Classifier(simple_featurize, L2_regularization_strength=1.0, min_feature_count=1)
simple_classifier.evaluate()

Sparse Matrix dimensions during traininig: (1000, 2)
Sparse Matrix dimensions during traininig: (1000, 2)
Sparse Matrix dimensions during traininig: (2000, 2)
Method: simple_featurize, Features: 2, Train accuracy: 0.509, Dev accuracy: 0.500


In [195]:
# ### Understanding the class Classifier
def load_data(filename=trainingFile):
    data = []
    with open(filename, encoding="utf8") as file:
        for line in file:
            cols = line.split("\t")
            idd = cols[0]
            label = cols[1]
            text = cols[2]

            data.append((idd, label, text))

    return data

# train_data = load_data()
# print(f"len of train_data: {len(train_data)}")

# ev_data = load_data(filename=evaluationFile)
# print(f"len of load_data: {len(ev_data)}")

test_data = load_data(filename=testFile)
print(f"len of test_data: {len(test_data)}")


# # playing around with nltk.featurize_word:
# print(train_data[0][2])
# words = nltk.word_tokenize(train_data[0][2])
# print(words)

len of test_data: 2000


In [198]:
test_data[1]

('6678',
 '',
 "Despite some reviews being distinctly Luke-warm, I found the story totally engrossing and even if some critics have described the love story as 'Mills and Boon', so what? It is good to see a warm, touching story of real love in these cynical times. Many in the audience were sniffing and surreptitiously dabbing their eyes. You really believe that the young Victoria and Albert are passionately fond of each other, even though, for political reasons, it was an arranged marriage. I did feel though that Sir John Conroy, who was desperate to control the young Queen, is perhaps played too like a pantomime villain. As it is rumoured that he was in fact, the real father of Victoria (as a result of an affair with her mother The Duchess of Kent) it would have been interesting to explore this theory. Emily Blunt is totally convincing as the young Princess, trapped in the stifling palace with courtiers and politicians out to manipulate her. She brilliantly portrays the strength of ch

So we've created a classifier. But is its accuracy score any good?  Let's calculate the accuracy of a "majority classifier" to provide some context. This determines the most-represented (majority) class in the training data, and then predicts every test point to be this class.

In [71]:
def majority_class(trainY, devY):
    labelCounts=Counter()
    for label in trainY:
        labelCounts[label]+=1
    majority_class=labelCounts.most_common(1)[0][0]

    correct=0.
    for label in devY:
        if label == majority_class:
            correct+=1

    print("Majority class: %s\tDev accuracy: %.3f" % (majority_class, correct/len(devY)))
majority_class(simple_classifier.trainY, simple_classifier.devY)

Majority class: pos	Dev accuracy: 0.500


The feature we created in `simple_featurize`, evidently, doesn't have a whole lot of legs. In the next portion of the homework, you'll be designing a few features of your own in the hopes of achieving the highest accuracy possible.

## Deliverable 1: Exploring Features (60 points)

Your job in this homework is to implement a binary bag-of-words model (i.e., one that assigns a feature value of 1 to each word type that is present in the text); and to brainstorm **3 additional** distinct classes of features, justify why they might help improve the performance *over a bag of words* for this task, implement them in code, and then assess their independent performance on the development data.

To show your work: describe your new features and report their performance in the table below; implement the features in the specified `feature1`, `feature2`, and `feature3` functions, and execute each respective classifier to show its performance. **Double click to edit the table below in markdown mode**.

|Feature|Why should it work? (50 words each)|Dev set performance|
|---|---|---|
|Bag of words||
|Feature 1||
|Feature 2||
|Feature 3||

**You are not graded on your performance, but your justification for why it *should* perform better than a bag of words should be defensible**. Consider the type of data you are working with: what do you look for when writing/reading a movie review?

### Implement Bag-of-Words

In [83]:
## A bag-of-words is a representation of text that describes the occurence of words within a document. It involves two things:
## - a vocabulary of known words
## - a measure of the presence of known words
## Any information on the order or structure of words in the document is discarded (similar documents have similar contents)
##

######################################################################
## For `bag_of_words`, please do not use external packages/libraries
## Only excepted external libraries are:
## - nltk
######################################################################

def bag_of_words(text):
    # Here the `feats` dict should contain the features -- the key should be the feature name,
    # and the value is the feature value.  See `simple_featurize` for an example.

    # BEGIN SOLUTION
    feats = {}
    words = nltk.word_tokenize(text)

    for word in words:
        word=word.lower()

        if word not in feats:
          feats[word] = 1

    # END SOLUTION

    return feats

In [91]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################

bow_classifier = Classifier(bag_of_words, L2_regularization_strength=1.0, min_feature_count=1)
bow_classifier.evaluate()

Method: bag_of_words, Features: 21078, Train accuracy: 1.000, Dev accuracy: 0.777


### Implement Your Original Features

**Note**: External dictionaries such as [AFINN](https://pypi.org/project/afinn) or [LIWC](https://pypi.org/project/liwc/) might be helpful here.

In [102]:
def feature1(text):
    # Here the `feats` dict should contain the features -- the key should be the feature name,
    # and the value is the feature value.  See `simple_featurize` for an example.

    # feature1 is a simple frequency based bag-of-words approach where the number of occurences of each word in an example is
    # recorded as opposed to the binary bag-of-words approach that simply records the occurence of a word in the whole example.


    feats = {}

    # BEGIN SOLUTION
    words = nltk.word_tokenize(text)

    for word in words:
        word=word.lower()

        if word not in feats:
          feats[word] = 1
        else:
          feats[word] += 1

    # END SOLUTION

    return feats

In [103]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################

classifier1 = Classifier(feature1, L2_regularization_strength=1.0, min_feature_count=1)
classifier1.evaluate()

Method: feature1, Features: 21078, Train accuracy: 1.000, Dev accuracy: 0.768


In [173]:
!pip install afinn
!pip install -U liwc

In [175]:
from afinn import Afinn
import liwc
path = "/content/drive/MyDrive/LIWC2007_English100131.dic"
parse, category_names = liwc.load_token_parser(path)


In [178]:
def feature2(text):
    # Here the `feats` dict should contain the features -- the key should be the feature name,
    # and the value is the feature value.  See `simple_featurize` for an example.

    feats = {}

    # BEGIN SOLUTION
    afn = Afinn()
    words = nltk.word_tokenize(text)
    sentences = nltk.sent_tokenize(text)
    pos, neg, neutral = 0,0,0

    overall = afn.score(text) # overall sentiment score from AFINN

    for sent in sentences:
      score = afn.score(sent)

      if score < 0:
        neg += 1
      elif score > 0:
        pos += 1
      else:
        neutral += 1

    text_counts = Counter(category for token in words for category in parse(token))

    #feats['AFINN Overall'] = overall
    feats['AFINN Positive'] = pos
    feats['AFINN Negative'] = neg
    feats['AFINN Neutral'] = neutral


    for liwc_cat in text_counts.keys():
      feats[liwc_cat] = text_counts[liwc_cat]

    # END SOLUTION
    return feats

In [179]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################

classifier2 = Classifier(feature2, L2_regularization_strength=1.0, min_feature_count=1)
classifier2.evaluate()

Method: feature2, Features: 67, Train accuracy: 0.779, Dev accuracy: 0.775


In [180]:
from nltk import ngrams

In [203]:
def feature3(text):
    # Here the `feats` dict should contain the features -- the key should be the feature name,
    # and the value is the feature value.  See `simple_featurize` for an example.

    feats = {}
    # BEGIN SOLUTION
    afn = Afinn()
    highpos, pos, neutral, neg, highneg = 0,0,0,0,0
    sent_pos, sent_neg, sent_neutral = 0,0,0
    words = nltk.word_tokenize(text)

    trigrams = ngrams(words,3)
    sentences = nltk.sent_tokenize(text)

    for sent in sentences:
      score = afn.score(sent)

      if score > 0:
        sent_pos += 1
      elif score < 0:
        sent_neg += 1
      else:
        sent_neutral += 1

    feats['AFINN Positive'] = pos
    feats['AFINN Negative'] = neg
    feats['AFINN Neutral'] = neutral

    for checking in trigrams:
      score = afn.score(''.join(checking)) # overall sentiment score from AFINN

      if score >= 2.5:
        highpos += 1
      elif score < 2.5 and score > 0:
        pos += 1
      elif score < 0 and score >= -2.5:
        neg += 1
      elif score < -2.5:
        highneg += 1
      else:
        neutral += 1

    feats['AFINN High Positive ngram'] = highpos
    feats['AFINN Positive ngram'] = pos
    feats['AFINN Neutral ngram'] = neutral
    feats['AFINN Negative ngram'] = neg
    feats['AFINN High Negative ngram'] = highneg

    # END SOLUTION
    return feats

In [204]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################

classifier3 = Classifier(feature3, L2_regularization_strength=1.0, min_feature_count=1)
classifier3.evaluate()

Method: feature3, Features: 8, Train accuracy: 0.650, Dev accuracy: 0.668


## Deliverable 2: Final Classifier (20 points)

The two cells in "Combine your features" will generate a file named `combiner_function_predictions.csv`. **Download this file (using e.g. the file manager on the left panel in Colab) and submit this to GradeScope along with your notebook**

Your score is computed by the following equation: `min(20.0, 200 * max(test_accuracy - 0.69, 0.0))`. **That is, you receive full credits with test accuracy >= 0.79 and no credits with test accuracy <= 0.69**.

The 5 students with the highest performance (revealed after the submission deadline) will receive extra credit for this assignment.

Please do not change the auto-generated filename!

### Combine your features

Next, let's combine any or all the features you have developed into one big model and make predictions on the test data. There is no exact number/threshold we're looking for, accuracy-wise, but the combiner function should *generally* have a higher accuracy than BoW on its own (assuming your features are adding additional information beyond what BoW is adding).

You don't need to edit the following cell, unless you want to change which features are handed off to the "combined" model.

In [201]:
def combiner_function(text):

    # Here the `all_feats` dict should contain the features -- the key should be the feature name,
    # and the value is the feature value.  See `simple_featurize` for an example.
    # at the moment, all 4 of: bag of words and your 3 original features are handed off to the combined model
    # update the values within [bag_of_words, feature1, feature2, feature3] to change this.

    all_feats={}
    for feature in [bag_of_words, feature1, feature2, feature3]:
        all_feats.update(feature(text))
    return all_feats

In [202]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################

big_classifier = Classifier(combiner_function, L2_regularization_strength=1.0, min_feature_count=1)
big_classifier.evaluate()

#generate .csv file with prediction output on test data
big_classifier.predict()

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Method: combiner_function, Features: 21111, Train accuracy: 1.000, Dev accuracy: 0.801


## Deliverable 3: Analyze and Tune Your Classifier (5 points)


Below you will find several ways in which you can interrogate your model to get ideas on ways to improve its performance. **Section 1 to 3 are for exploration. The only deliverable is in section 4.**

### 1. Confusion Matrix

First, let's look at the confusion matrix of its predictions (where we can compare the true labels with the predicted labels). What kinds of mistakes is it making? (While this is mainly helpful in the context of multiclass classification, we can still see if there's a bias toward predicting a specific class in the binary setting as well). **No written response required.**

In [ ]:
def print_confusion(classifier):
    fig, ax = plt.subplots(figsize=(10,10))

    # Compute the confusion matrix
    y_pred = classifier.log_reg.predict(classifier.devX)
    cm = confusion_matrix(classifier.devY, y_pred)

    # Use ConfusionMatrixDisplay to plot
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=ax, xticks_rotation="vertical", values_format="d")

    plt.show()

print_confusion(big_classifier)


### 2. Feature Weights

Next, let's look at the features that are most important for each of the classes (ranked by how strong their corresponding coefficient is). Do the features you are defining help in the ways you think they should?  Do sets of successful features suggests others, or complementary features that may provide a different view on the data? **No written response required.**

In [ ]:
big_classifier.printWeights(n=25)

### 3. Misclassified Data Points

Next, let's look at the individual data points that are the hardest to classify correctly. Does it suggest any features you might create to disentangle them? **No written response required.**

In [ ]:
def analyze(classifier):

    probs=classifier.log_reg.predict_proba(classifier.devX)
    predicts=classifier.log_reg.predict(classifier.devX)

    classes={}
    for idx, lab in enumerate(classifier.log_reg.classes_):
        classes[lab]=idx

    mistakes={}
    for i in range(len(probs)):
        if predicts[i] != classifier.devY[i]:
            predicted_lab_idx=classes[predicts[i]]
            mistakes[i]=probs[i][predicted_lab_idx]

    frame=[]
    sorted_x = sorted(mistakes.items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x:
        idd=classifier.devOrig[k][0]
        text=classifier.devOrig[k][2]
        frame.append([idd, v, classifier.devY[k], predicts[k], text])

    df=pd.DataFrame(frame, columns=["id", "P(predicted class confidence)", "Human label", "Prediction", "Text"])

    with option_context('display.max_colwidth', 400):
        display(df.head(n=20))

analyze(big_classifier)

### 4. Hyperparameter Tuning

An important step in optimizing models is hyperparameter tuning. Hyperparameter tuning enables us to adjust the hyperparameters of our model, specifically those not learned from the data, to optimize its performance. Below, you will use K-fold cross-validation on the training data to get an estimate of your model's performance. For this deliverable, **replace `classifier` with the classifier you want to improve. Then, complete `param_grid` for `C` (values closer to 0 = stronger regularization)** to find which L2 regularization strength improves your model the most. For more information, please consult [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

*Note: we are unable to hyperparameter tune `min_feature_size` easily with the scikitlearn API (we hard coded `min_feature_size` in the `Classifier` class). Please try out different values manually.*

In [ ]:
# import libraries
from sklearn.model_selection import KFold, GridSearchCV

# Parameter Grid for L2 regularization strength
# BEGIN SOLUTION
param_grid = {'C': ...}
# END SOLUTION

# initialize KFold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

classifier = big_classifier # TODO: Change to the classifier you want to hyperparameter tune

grid_search = GridSearchCV(estimator= classifier.log_reg, param_grid=param_grid, scoring='accuracy', cv=kf)
grid_search.fit(classifier.trainX, classifier.trainY)
best_params = grid_search.best_params_

# Which parameters performed best?
print(best_params)

## Deliverable 4: Short Answer (15 points)

Give a real-world example of where it is important to capture the percentage of false positives the model predicts and another example of a case where it is important to capture false negatives. Why might these measures be more valuable than finding the model's accuracy? (~50 words)

*Double click to edit*